<a href="https://colab.research.google.com/github/wenqiglantz/llmops/blob/main/Eval_for_quantized_models_for_Mistral_7B_Instruct_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluate the quantized Mistral-7B-Instruct-v0.2 using LlamaIndex's rag_evaluator LlamaPack

This notebook demonstrates how to evaluate the quantized models for `Mistral-7B-Instruct-v0.2` and compare the performance between the two quantized models.


In [ ]:
!pip install llama_index==0.9.25 llama_hub torch transformers accelerate bitsandbytes llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 80.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-hub to determine which version is com

In [ ]:
import logging, sys
import nest_asyncio

nest_asyncio.apply()

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [ ]:
from llama_index.llama_dataset import download_llama_dataset
from llama_index.llama_pack import download_llama_pack
from llama_index import VectorStoreIndex
from llama_index.llms import OpenAI
from google.colab import userdata
import os

# get the OpenAI API key from secrets tab in Colab
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

### Evaluate 5-bit quantized model

In [ ]:
from llama_index.llms import LlamaCPP
from llama_index import ServiceContext

# define llm by calling LlamaCPP, pass in the gguf file from hugging face hub
llm_q5 = LlamaCPP(
    model_url="https://huggingface.co/wenqiglantz/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf"
)

# define ServiceContext
service_context_q5 = ServiceContext.from_defaults(
    llm=llm_q5,
    embed_model="local:WhereIsAI/UAE-Large-V1"
)

total size (MB): 5132.35


4895it [00:30, 160.30it/s]                          
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


We download both the Llama dataset and RagEvaluatorPack. We use Paul Graham's essay dataset in our evaluation. From the dataset, the pack uses SimpleDirectoryReader to load the data into documents, and we then construct the VectorStoreIndex from the documents.

In [ ]:
# download and install RagEvaluatorPack
RagEvaluatorPack = download_llama_pack(
  "RagEvaluatorPack", "./rag_evaluator_pack"
)

# download a LabelledRagDataset from llama-hub
rag_dataset_q5, documents_q5 = download_llama_dataset(
    "PaulGrahamEssayDataset", "./paul_graham"
)

# build index from the source documents
index_q5 = VectorStoreIndex.from_documents(documents=documents_q5)

# define query engine
query_engine_q5 = index_q5.as_query_engine(service_context=service_context_q5)

# construct RagEvaluatorPack
rag_evaluator_pack_q5 = RagEvaluatorPack(
    query_engine=query_engine_q5,
    rag_dataset=rag_dataset_q5,
    judge_llm=OpenAI(temperature=0, model="gpt-4-1106-preview")
)

# run eval
benchmark_df_q5 = rag_evaluator_pack_q5.run()
print(benchmark_df_q5)

100%|██████████| 4/4 [22:55<00:00, 343.87s/it]
2it [00:13,  6.78s/it]
2it [00:17,  8.89s/it]
2it [00:17,  8.57s/it]
2it [00:09,  4.99s/it]
2it [00:17,  8.65s/it]
2it [00:16,  8.37s/it]
2it [00:14,  7.35s/it]
2it [00:14,  7.34s/it]
2it [00:15,  7.68s/it]
2it [00:13,  6.75s/it]
2it [00:22, 11.19s/it]
2it [00:16,  8.39s/it]
2it [00:17,  8.92s/it]
2it [00:12,  6.33s/it]
2it [00:24, 12.43s/it]
2it [00:15,  7.64s/it]
2it [00:14,  7.46s/it]
2it [00:15,  7.69s/it]
2it [00:16,  8.22s/it]
2it [00:20, 10.12s/it]
2it [00:14,  7.11s/it]
2it [00:17,  8.58s/it]


rag                            base_rag
metrics                                
mean_correctness_score         3.625000
mean_relevancy_score           0.590909
mean_faithfulness_score        1.000000
mean_context_similarity_score  0.932185


### Evaluate 4-bit quantized model

In [ ]:
from llama_index.llms import LlamaCPP
from llama_index import ServiceContext

llm_q4 = LlamaCPP(
    model_url="https://huggingface.co/wenqiglantz/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf"
)

service_context_q4 = ServiceContext.from_defaults(
    llm=llm_q4,
    embed_model="local:WhereIsAI/UAE-Large-V1"
)

total size (MB): 4369.38


4167it [00:17, 231.70it/s]                          
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# download and install RagEvaluatorPack
RagEvaluatorPack = download_llama_pack(
  "RagEvaluatorPack", "./rag_evaluator_pack"
)

# download a LabelledRagDataset from llama-hub
rag_dataset_q4, documents_q4 = download_llama_dataset(
    "PaulGrahamEssayDataset", "./paul_graham"
)

# build index from the source documents
index_q4 = VectorStoreIndex.from_documents(documents=documents_q4)

# define query engine
query_engine_q4 = index_q4.as_query_engine(service_context=service_context_q4)

# construct RagEvaluatorPack
rag_evaluator_pack_q4 = RagEvaluatorPack(
    query_engine=query_engine_q4,
    rag_dataset=rag_dataset_q4,
    judge_llm=OpenAI(temperature=0, model="gpt-4-1106-preview")
)

# run eval
benchmark_df_q4 = rag_evaluator_pack_q4.run()
print(benchmark_df_q4)

100%|██████████| 4/4 [19:36<00:00, 294.21s/it]
2it [00:17,  8.80s/it]
2it [00:18,  9.31s/it]
2it [00:16,  8.07s/it]
2it [00:17,  8.93s/it]
2it [00:21, 10.99s/it]
2it [00:22, 11.23s/it]
2it [00:18,  9.27s/it]
2it [00:16,  8.30s/it]
2it [00:11,  5.97s/it]
2it [00:20, 10.28s/it]
2it [00:25, 12.58s/it]
2it [00:16,  8.38s/it]
2it [00:29, 14.77s/it]
2it [00:17,  8.56s/it]
2it [00:23, 11.97s/it]
2it [00:20, 10.17s/it]
2it [00:27, 13.54s/it]
2it [00:23, 11.88s/it]
2it [00:26, 13.44s/it]
2it [00:16,  8.12s/it]
2it [00:20, 10.29s/it]
2it [00:20, 10.13s/it]


rag                            base_rag
metrics                                
mean_correctness_score         3.670455
mean_relevancy_score           0.681818
mean_faithfulness_score        0.977273
mean_context_similarity_score  0.932186
